## Final Capstone project

### Installing prerequisites

In [1]:
#importing required libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
#installing requests to work with URLs (if not installed before)
!pip install requests
#importing requests
import requests

#Importing additional dependencies to show neighbourhoods in the map
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Install and import completed')

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Install and import completed


In [2]:
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

### Define Foursquare Credentials

In [3]:
CLIENT_ID = '0GPMCSTZ5SOR20WJQNQWRXM0YHXQDMIKAQKHMTPQM2FPHPES' #Foursquare ID
CLIENT_SECRET = 'N2VIOQ1Y45VH2MCATYXQJQZS2AEALSIC4UPVWKUGBFU0GSZS' #Foursquare Secret
VERSION = '20181102' # Foursquare API version
LIMIT = 100

### In our project we are going to compare trending venues around Empire State Building (NewYork) and CN Tower (Toronto). Let's convert the Empire State Building and CN Tower address to their latitude and longitude coordinates.

In [4]:
ESB_address = 'Empire State Building, New York, NY'
CNT_address = 'CN Tower, Front Street West, Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(ESB_address)

ESB_latitude = location.latitude
ESB_longitude = location.longitude

location = geolocator.geocode(CNT_address)
CNT_latitude = location.latitude
CNT_longitude = location.longitude

print("Empire State Building: ", ESB_latitude, ESB_longitude)
print("CN Tower: ", CNT_latitude, CNT_longitude)

/anaconda3/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


Empire State Building:  40.7484284 -73.9856546198733
CN Tower:  43.6425637 -79.3870871832047


### Let's explore trending venues

 For trending venues Foursquare URL looks like that:
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

##### We are interested in knowing the venues that are trending at this exact time, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Empire State Building and CN Tower.

In [5]:
# define URLs
ESB_url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, ESB_latitude, ESB_longitude, VERSION)
CNT_url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, CNT_latitude, CNT_longitude, VERSION)

# send GET requests and get trending venues
ESB_results = requests.get(ESB_url).json()
CNT_results = requests.get(CNT_url).json()
print(ESB_results)
print("\n")
print(CNT_results)

{'meta': {'code': 200, 'requestId': '5bdc6b179fb6b71cdf6d0f2c'}, 'response': {'venues': [{'id': '42829c80f964a5206a221fe3', 'name': 'Grand Central Terminal', 'location': {'address': '87 E 42nd St', 'crossStreet': 'btwn Vanderbilt & Park Ave', 'lat': 40.752841608758914, 'lng': -73.97759371330933, 'distance': 838, 'postalCode': '10017', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['87 E 42nd St (btwn Vanderbilt & Park Ave)', 'New York, NY 10017', 'United States']}, 'categories': [{'id': '4bf58dd8d48988d129951735', 'name': 'Train Station', 'pluralName': 'Train Stations', 'shortName': 'Train Station', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/trainstation_', 'suffix': '.png'}, 'primary': True}], 'venuePage': {'id': '91385129'}}, {'id': '4283ee00f964a5209d221fe3', 'name': 'Jacob K. Javits Convention Center', 'location': {'address': '655 W 34th St', 'crossStreet': 'at 11th Ave', 'lat': 40.75714002927082, 'lng': -74.00

##### Check if any venues are trending at this time

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# function that check if venues exist
def checkVenues (json):
    results = json
    if len(results['response']['venues']) == 0:
        trending_venues_df = 'No trending venues are available at the moment!'
    
    else:
        trending_venues = results['response']['venues']
        trending_venues_df = json_normalize(trending_venues)

        # filter columns
        columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
        trending_venues_df = trending_venues_df.loc[:, columns_filtered]

        # filter the category for each row
        trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)
    return trending_venues_df

In [7]:
ESB_trending_venues_df = checkVenues(ESB_results)
CNT_trending_venues_df = checkVenues(CNT_results)

In [8]:
#look at ESB trending venues
ESB_trending_venues_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Grand Central Terminal,Train Station,838,New York,10017,NY,United States,40.752842,-73.977594
1,Jacob K. Javits Convention Center,Convention Center,1713,New York,10001,NY,United States,40.757140,-74.002404
2,New York Penn Station,Train Station,680,New York,10001,NY,United States,40.750687,-73.993152
3,Google New York,Office,1715,New York,10011,NY,United States,40.741616,-74.003906
4,Museum of Modern Art (MoMA),Art Museum,1625,New York,10019,NY,United States,40.761531,-73.977135
5,Port Authority Bus Terminal,Bus Station,1093,New York,10018,NY,United States,40.756905,-73.992200
6,Foursquare HQ,Tech Startup,842,New York,10010,NY,United States,40.742320,-73.991562


In [9]:
#look at CNT trending venues
CNT_trending_venues_df

'No trending venues are available at the moment!'

##### Lets save resulting venues (if any) in file adding a timestamp, so we can review it later

In [10]:
#import datetime for timestamp
from datetime import datetime

In [11]:
#define function to check for venues and save to file
def saveVenues (trending_venues_df, name):
    #if it's a string (no venues), don't save
    if isinstance(trending_venues_df, str):
        print("There is no venues at current time")
    #otherwise (if dataframe) save to file with timestamp
    else:
        filename = name + datetime.now().strftime('%Y%m%d%H%M%S')
        trending_venues_df.to_csv(filename)
        print("File was saved to ", filename)

In [12]:
#attempt to save 
saveVenues(CNT_trending_venues_df, "CNT")
saveVenues(ESB_trending_venues_df, "ESB")

There is no venues at current time
File was saved to  ESB20181102112106


##### we can check if file was correctly saved

In [13]:
checktmp = pd.read_csv("ESB20181102112106").drop(['Unnamed: 0'],axis=1)
checktmp.head()

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Grand Central Terminal,Train Station,838,New York,10017,NY,United States,40.752842,-73.977594
1,Jacob K. Javits Convention Center,Convention Center,1713,New York,10001,NY,United States,40.757140,-74.002404
2,New York Penn Station,Train Station,680,New York,10001,NY,United States,40.750687,-73.993152
3,Google New York,Office,1715,New York,10011,NY,United States,40.741616,-74.003906
4,Museum of Modern Art (MoMA),Art Museum,1625,New York,10019,NY,United States,40.761531,-73.977135


##### We can also Visualize trending venues

In [14]:
#define visualize function:
def visualVenues(trending_venues_df, latitude, longitude):
    if isinstance(trending_venues_df, str):
        venues_map = 'Cannot generate visual as no trending venues are available at the moment!'
    else:
        venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around selected location

    # add our location as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
    return venues_map

In [15]:
early_venues_map = visualVenues(ESB_trending_venues_df, ESB_latitude, ESB_longitude)

In [16]:
early_venues_map

### Now, after a while, we can check trending venues again. They should be different from previous because we call it in different time.

In [17]:
ESB_results = requests.get(ESB_url).json()
CNT_results = requests.get(CNT_url).json()

In [18]:
ESB_trending_venues_df = checkVenues(ESB_results)
CNT_trending_venues_df = checkVenues(CNT_results)

In [19]:
ESB_trending_venues_df

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Jacob K. Javits Convention Center,Convention Center,1713,New York,10001,NY,United States,40.757140,-74.002404
1,Museum of Modern Art (MoMA),Art Museum,1625,New York,10019,NY,United States,40.761531,-73.977135
2,New York Penn Station,Train Station,680,New York,10001,NY,United States,40.750687,-73.993152
3,Grand Central Terminal,Train Station,838,New York,10017,NY,United States,40.752842,-73.977594
4,Google New York,Office,1715,New York,10011,NY,United States,40.741616,-74.003906
5,Shake Shack,Burger Joint,819,New York,10010,NY,United States,40.741353,-73.988337
6,Turntable Chicken Jazz,Fried Chicken Joint,54,New York,10001,NY,United States,40.748428,-73.986302
7,Xi'an Famous Foods,Chinese Restaurant,879,New York,10010,NY,United States,40.740632,-73.987346


In [20]:
CNT_trending_venues_df

'No trending venues are available at the moment!'

In [21]:
saveVenues(CNT_trending_venues_df, "CNT")
saveVenues(ESB_trending_venues_df, "ESB")

There is no venues at current time
File was saved to  ESB20181102132650


### Finally we can compare trending venues in after some time passed

In [22]:
early_venues = pd.read_csv("ESB20181102112106").drop(['Unnamed: 0'],axis=1)
early_venues

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Grand Central Terminal,Train Station,838,New York,10017,NY,United States,40.752842,-73.977594
1,Jacob K. Javits Convention Center,Convention Center,1713,New York,10001,NY,United States,40.757140,-74.002404
2,New York Penn Station,Train Station,680,New York,10001,NY,United States,40.750687,-73.993152
3,Google New York,Office,1715,New York,10011,NY,United States,40.741616,-74.003906
4,Museum of Modern Art (MoMA),Art Museum,1625,New York,10019,NY,United States,40.761531,-73.977135
5,Port Authority Bus Terminal,Bus Station,1093,New York,10018,NY,United States,40.756905,-73.992200
6,Foursquare HQ,Tech Startup,842,New York,10010,NY,United States,40.742320,-73.991562


In [23]:
later_venues = pd.read_csv("ESB20181102132650").drop(['Unnamed: 0'],axis=1)
later_venues

,name,categories,location.distance,location.city,location.postalCode,location.state,location.country,location.lat,location.lng
0,Jacob K. Javits Convention Center,Convention Center,1713,New York,10001,NY,United States,40.757140,-74.002404
1,Museum of Modern Art (MoMA),Art Museum,1625,New York,10019,NY,United States,40.761531,-73.977135
2,New York Penn Station,Train Station,680,New York,10001,NY,United States,40.750687,-73.993152
3,Grand Central Terminal,Train Station,838,New York,10017,NY,United States,40.752842,-73.977594
4,Google New York,Office,1715,New York,10011,NY,United States,40.741616,-74.003906
5,Shake Shack,Burger Joint,819,New York,10010,NY,United States,40.741353,-73.988337
6,Turntable Chicken Jazz,Fried Chicken Joint,54,New York,10001,NY,United States,40.748428,-73.986302
7,Xi'an Famous Foods,Chinese Restaurant,879,New York,10010,NY,United States,40.740632,-73.987346


###### As you can see there are differences that I'll talk about in Final report

### Also let's visualize later set of venues

In [24]:
early_venues_map = visualVenues(ESB_trending_venues_df, ESB_latitude, ESB_longitude)

In [25]:
early_venues_map

##### We will discuss all the observations in final report as per assignment